In [1]:
!pip install opencv-python-headless==4.8.0.74 openmim==0.1.5 timm==0.5.4 torch==1.11.0 torchvision==0.12.0
# !pip install opencv-python-headless openmim timm torch torchvision

  Preparing metadata (setup.py) ... done
Reason for being yanked: deprecated, use 4.8.0.76
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.5/431.5 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.6/750.6 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 6.2 MB/s eta 0:00:00
  Created wheel for openmim: filename=openmim-0.1.5-py2.py3-none-any.whl size=42472 sha256=4fdb5b6cf1372fc3aea0faceb4949ac933859a713563d4a508bc904d1822446e
  Stored in directory: /root/.cache/pip/wheels/33/51/69/7d124df565cadec731be86337a2e147e024e8eba8fab493182
Successfully built openmim
  Attempting uninstall: torch
    Found existing installation: torch 2.4.1+cu121
    Uninstalling torch-2.4.1+cu121:
      Successfully uninstalled torch-2.4.1+cu121
  Attempting uninstall: opencv-python-headless
    Found 

In [2]:
!pip install yapf==0.31.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.7/185.7 kB 11.3 MB/s eta 0:00:00


In [3]:
!mim install mmcv-full==1.5.0
!pip install mmdet==2.24.1

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
installing mmcv-full from wheel.
Looking in links: https://download.openmmlab.com/mmcv/dist/cpu/torch1.11.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 37.9 MB/s eta 0:00:00


In [4]:
!git clone https://github.com/ViTAE-Transformer/ViTPose.git

Cloning into 'ViTPose'...
remote: Enumerating objects: 1859, done.
remote: Counting objects: 100% (1041/1041), done.
remote: Compressing objects: 100% (316/316), done.
remote: Total 1859 (delta 794), reused 725 (delta 725), pack-reused 818 (from 1)
Receiving objects: 100% (1859/1859), 10.75 MiB | 28.73 MiB/s, done.
Resolving deltas: 100% (958/958), done.


In [ ]:
# !mim install mmdet==2.24.1 mmpose
# !mim install mmdet
#↑これをやるとmmcv==2.1.0のインストールを始めて、やたら時間がかかる。その後エラー
#先にmmcvをインストールする必要があるのかも
# !pip install mmdet==2.24.1 mmpose

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 4.0 MB/s eta 0:00:00
  Attempting uninstall: mmdet
    Found existing installation: mmdet 3.3.0
    Uninstalling mmdet-3.3.0:
      Successfully uninstalled mmdet-3.3.0


In [5]:
!mim download mmdet --config yolox_tiny_8x8_300e_coco --dest /content

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
processing yolox_tiny_8x8_300e_coco...
  [####################################]  100%          
Successfully downloaded yolox_tiny_8x8_300e_coco_20211124_171234-b4047906.pth to /content
Successfully dumped yolox_tiny_8x8_300e_coco.py to /content


In [6]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
from mmdet.apis import inference_detector, init_detector
import urllib.request

# 物体検出モデルの初期化
class DetModel:

    def __init__(self):
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.model = self._load_model()

    def _load_model(self):
        config_path = "/content/yolox_tiny_8x8_300e_coco.py"
        model_path = "/content/yolox_tiny_8x8_300e_coco_20211124_171234-b4047906.pth"

        return init_detector(config_path, model_path, device=self.device)

    def detect(self, image: np.ndarray):
        image = image[:, :, ::-1]  # RGB -> BGR
        out = inference_detector(self.model, image)
        return out

    def visualize_detection_results(self, image: np.ndarray, detection_results: list, score_threshold: float = 0.3):
        person_det = [detection_results[0]] + [np.array([]).reshape(0, 5)] * 79
        image = image[:, :, ::-1]  # RGB -> BGR
        vis = self.model.show_result(
            image, person_det, score_thr=score_threshold, bbox_color=None, text_color=(200, 200, 200), mask_color=None
        )
        return vis[:, :, ::-1]  # BGR -> RGB



det_model = DetModel()

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.16 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


load checkpoint from local path: /content/yolox_tiny_8x8_300e_coco_20211124_171234-b4047906.pth


In [9]:
!git clone https://github.com/open-mmlab/mmpose.git

Cloning into 'mmpose'...
remote: Enumerating objects: 31261, done.
remote: Counting objects: 100% (363/363), done.
remote: Compressing objects: 100% (234/234), done.
remote: Total 31261 (delta 174), reused 266 (delta 127), pack-reused 30898 (from 1)
Receiving objects: 100% (31261/31261), 54.31 MiB | 24.36 MiB/s, done.
Resolving deltas: 100% (22001/22001), done.


In [7]:
!mim install mmpose

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
mmpose==1.3.2 existed.


In [9]:
!mim install mmengine

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
installing mmengine from wheel.


In [5]:
!mim install mmpretrain>=1.0.0

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


In [3]:
!pip install mmpose
# !pip install mmpose==0.29.0

  Cloning https://github.com/jin-s13/xtcocoapi to /tmp/pip-install-vliyoycw/xtcocotools_c0da7d1cf4c64300b658a8bf244c4b30
  Running command git clone --filter=blob:none --quiet https://github.com/jin-s13/xtcocoapi /tmp/pip-install-vliyoycw/xtcocotools_c0da7d1cf4c64300b658a8bf244c4b30
  Resolved https://github.com/jin-s13/xtcocoapi to commit d74033ff1635e9002133b2380862bc2b728584d2
  Preparing metadata (setup.py) ... done


In [11]:
import sys
sys.path.append('/content/mmpose')


In [12]:
from mmpose.apis import inference_top_down_pose_model, init_pose_model, process_mmdet_results, vis_pose_result

# 姿勢推定モデルの初期化
class PoseModel:
    def __init__(self):
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.model = self._load_model()

    def _load_model(self):
        ckpt_path = "https://download.openmmlab.com/mmpose/v1/body_2d_keypoint/topdown_heatmap/coco/td-hm_ViTPose-small_8xb64-210e_coco-256x192-62d7a712_20230314.pth"
        config_path = "/content/mmpose/configs/body_2d_keypoint/topdown_heatmap/coco/td-hm_ViTPose-base-simple_8xb64-210e_coco-256x192.py"

        model = init_pose_model(config_path, ckpt_path, device=self.device)
        return model

    def predict_pose(self, image: np.ndarray, det_results: list, box_score_threshold: float = 0.5):
        image = image[:, :, ::-1]  # RGB -> BGR
        person_results = process_mmdet_results(det_results, 1)
        out, _ = inference_top_down_pose_model(
            self.model, image, person_results=person_results, bbox_thr=box_score_threshold, format="xyxy"
        )
        return out

    def visualize_pose_results(self, image: np.ndarray, pose_results: list, kpt_score_threshold: float = 0.3, vis_dot_radius: int = 4, vis_line_thickness: int = 1):
        image = image[:, :, ::-1]  # RGB -> BGR
        vis = vis_pose_result(
            self.model,
            image,
            pose_results,
            kpt_score_thr=kpt_score_threshold,
            radius=vis_dot_radius,
            thickness=vis_line_thickness,
        )
        return vis[:, :, ::-1]  # BGR -> RGB

pose_model = PoseModel()

ModuleNotFoundError: No module named 'mmpose.apis'

In [ ]:
# 画像のアップロード
uploaded = files.upload()

for filename in uploaded.keys():
    image_path = filename

# 画像の読み込み
image = plt.imread(image_path)

In [ ]:
# 物体検出
det_results = det_model.detect(image)
det_visualization = det_model.visualize_detection_results(image, det_results)

# 検出結果の表示
plt.figure(figsize=(10, 10))
plt.imshow(det_visualization)
plt.title("Detection Results")
plt.axis("off")
plt.show()

In [ ]:
# 姿勢推定
pose_results = pose_model.predict_pose(image, det_results)
pose_visualization = pose_model.visualize_pose_results(image, pose_results)

# 姿勢推定結果の表示
plt.figure(figsize=(10, 10))
plt.imshow(pose_visualization)
plt.title("Pose Estimation Results")
plt.axis("off")
plt.show()